In [53]:
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.sparse as sparse
import contextily as cx
import descartes

# Data curation
## Importing and wrangling spatial data

In [55]:
data = geopandas.read_file("metro-data.geojson")
sdf = geopandas.GeoDataFrame(data)

#Select only the columns of interest
sdf = sdf[['HMGNS_LNK_', 'geometry']]

#Change projection and rename a column
sdf = sdf.to_crs(epsg=7899)
sdf.columns = ["HMGNS_LNK_ID", "geometry"]

#Lines have duplicates (each for 2 sides of the road) - we're keeping only one
sdf = sdf.drop_duplicates(subset="HMGNS_LNK_ID", keep="first")

#sort by ID
sdf = sdf.sort_values("HMGNS_LNK_ID")
print(sdf.head(30))

      HMGNS_LNK_ID                                           geometry
173              2  MULTILINESTRING ((2512267.691 2400818.287, 251...
2756            17  MULTILINESTRING ((2502920.708 2425051.840, 250...
2865            25  MULTILINESTRING ((2497597.398 2404261.598, 249...
359             28  MULTILINESTRING ((2490756.645 2406178.082, 249...
323             30  MULTILINESTRING ((2502285.541 2399882.953, 250...
687             31  MULTILINESTRING ((2502042.935 2398281.947, 250...
4432            32  MULTILINESTRING ((2493520.307 2418368.862, 249...
4932            33  MULTILINESTRING ((2493690.510 2418294.537, 249...
4113            34  MULTILINESTRING ((2493756.538 2418264.670, 249...
3472            35  MULTILINESTRING ((2493800.541 2418245.546, 249...
3570            36  MULTILINESTRING ((2494058.222 2418143.065, 249...
360             37  MULTILINESTRING ((2480070.705 2413652.088, 248...
324             38  MULTILINESTRING ((2517332.156 2416836.598, 251...
688             39  

## Importing and cleaning data on traffic volume

In [48]:
dataset = pd.read_csv('TYPICAL_HOURLY_VOLUME_DATA.csv')
dataset = dataset.drop(['HMGNS_FLOW_ID', 'HMGNS_LNK_DESC', 'FLOW', 'COUNT_TYPE'], axis=1)
dataset = dataset[dataset['PERIOD_TYPE']=='SCHOOL TERM/NORMAL']

dataset = dataset.rename(columns={"0:00": "0", "1:00": "1", "2:00": "2", "3:00": "3", "4:00": "4", "5:00": "5", "6:00": "6", "7:00": "7", "8:00": "8", "9:00": "9", "10:00": "10", "11:00": "11", "12:00": "12", "13:00": "13", "14:00": "14", "15:00": "15", "16:00": "16", "17:00": "17", "18:00": "18", "19:00": "19", "20:00": "20", "21:00": "21", "22:00": "22", "23:00": "23"}, errors="raise")
dataset = dataset.drop(['PERIOD_TYPE'], axis=1)
dataset = dataset.melt(id_vars=['HMGNS_LNK_ID','DOW'], value_vars=dataset.iloc[:,2:27], var_name='Hour', value_name='Value')
dataset['Hour'] = pd.to_numeric(dataset['Hour'])
dataset = dataset.drop_duplicates(subset=["HMGNS_LNK_ID",'DOW','Hour'], keep="first")

dataset = dataset.sort_values(["HMGNS_LNK_ID",'DOW','Hour'])

#Normalising the traffic volume between 0 and 3 in order to plot it as linewidth
dataset['Normalised Value']=(dataset['Value']-dataset['Value'].min())/(dataset['Value'].max()-dataset['Value'].min())
dataset['Normalised Value']=3*dataset['Normalised Value']
print(dataset.head(25))

         HMGNS_LNK_ID  DOW  Hour  Value  Normalised Value
1974                2    1     0    886          0.245067
77023               2    1     1    489          0.135257
152072              2    1     2    335          0.092661
227121              2    1     3    317          0.087682
302170              2    1     4    468          0.129449
377219              2    1     5   1144          0.316430
452268              2    1     6   3177          0.878757
527317              2    1     7   5536          1.531256
602366              2    1     8   6322          1.748663
677415              2    1     9   4746          1.312742
752464              2    1    10   4434          1.226443
827513              2    1    11   4645          1.284805
902562              2    1    12   4985          1.378849
977611              2    1    13   5268          1.457127
1052660             2    1    14   6304          1.743684
1127709             2    1    15   7895          2.183754
1202758       

## Further wrangling of the traffic data

In [47]:
#Old version
#hourly = pd.read_csv('TYPICAL_HOURLY_VOLUME_DATA.csv')

#Dropping columns of no interest
#hourly_dropped = hourly.drop(['HMGNS_FLOW_ID', 'HMGNS_LNK_DESC', 'FLOW', 'COUNT_TYPE', 'PERIOD_TYPE'], axis=1)

#Renaming the 'hours' columns to be able to sort them from 0 to 23
#hourly_dropped = hourly_dropped.rename(columns={"0:00": "0", "1:00": "1", "2:00": "2", "3:00": "3", "4:00": "4", "5:00": "5", "6:00": "6", "7:00": "7", "8:00": "8", "9:00": "9", "10:00": "10", "11:00": "11", "12:00": "12", "13:00": "13", "14:00": "14", "15:00": "15", "16:00": "16", "17:00": "17", "18:00": "18", "19:00": "19", "20:00": "20", "21:00": "21", "22:00": "22", "23:00": "23"}, errors="raise")

#Melting the data to make it long
#hourly_long = hourly_dropped.melt(id_vars=['HMGNS_LNK_ID','DOW'], value_vars=hourly_dropped.iloc[:,2:27], var_name='Hour', value_name='Value')
#hourly_long['Hour'] = pd.to_numeric(hourly_long['Hour'])
#sorted_df = hourly_long.sort_values(['HMGNS_LNK_ID','DOW','Hour'])
#print(sorted_df.head(5))
#Grouping by: road ID, Day of week and Hour and calculating average value of traffic volume for each
#grouped_multiple = hourly_long.groupby(['HMGNS_LNK_ID', 'DOW', 'Hour']).agg({'Value': 'mean'})
#grouped_multiple.columns = ['Average Value']
#grouped_multiple = grouped_multiple.reset_index()
#Sorting by Road ID, Day of week and Hour
#sorted_df = grouped_multiple.sort_values(["HMGNS_LNK_ID"])
#print(sorted_df.head(170))

### Combining spatial data with the traffic volume data by road ID


In [56]:
df = sdf.merge(dataset, on="HMGNS_LNK_ID", how="right")
print(df.head(25))

    HMGNS_LNK_ID                                           geometry  DOW  \
0              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
1              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
2              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
3              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
4              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
5              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
6              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
7              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
8              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
9              2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
10             2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
11             2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1   
12          

# Plotting

In [58]:
#Sorting by Road ID, Day of week and Hour
df = df.sort_values(["HMGNS_LNK_ID", 'DOW', 'Hour'])

print(df.head())

   HMGNS_LNK_ID                                           geometry  DOW  Hour  \
0             2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1     0   
1             2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1     1   
2             2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1     2   
3             2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1     3   
4             2  MULTILINESTRING ((2512267.691 2400818.287, 251...    1     4   

   Value  Normalised Value  
0    886          0.245067  
1    489          0.135257  
2    335          0.092661  
3    317          0.087682  
4    468          0.129449  


In [67]:
for dow in df['DOW']:
    day_df = df[df['DOW']==dow]
    print("It is day ", dow)
    for hour in day_df['Hour']:
        #day_hour_df = df[df['Hour'] == hour]
        print("It is day ", dow, " and hour is ", hour)

It is day  1
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  an

It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  

It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and ho

It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  

It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is 

...

It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is 

It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour i

It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and ho

It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10
It is day  1  and hour is  11
It is day  1  and hour is  12
It is day  1  and hour is  13
It is day  1  and hour is  14
It is day  1  and hour is  15
It is day  1  and hour is  16
It is day  1  and hour is  17
It is day  1  and hour is  18
It is day  1  and hour is  19
It is day  1  and hour is  20
It is day  1  and hour is  21
It is day  1  and hour is  22
It is day  1  and hour is  23
It is day  1  and hour is  0
It is day  1  and hour is  1
It is day  1  and hour is  2
It is day  1  and hour is  3
It is day  1  and hour is  4
It is day  1  and hour is  5
It is day  1  and hour is  6
It is day  1  and hour is  7
It is day  1  and hour is  8
It is day  1  and hour is  9
It is day  1  and hour is  10

KeyboardInterrupt: 

### Basemaps

In [ ]:
print(cx.providers.keys())
cx.providers.CartoDB.keys()

    for hour in day_df['Hour']:
        day_hour_df = day_df[day_df['Hour']==hour]
        day_hour_df.plot(ax=ax, linewidth=day_hour_df['Normalised Value'])
        ax.axis('off')
        plt.savefig("day{}_hour{}.png".format(dow, hour), dpi = 200, facecolor='#333333')